# Week 2 - Overview of Lyapunov Stability Theory

**<ins>Motivation</ins>**

In this week, we introduce **Lyapunov’s Direct Method** as a powerful tool to analyze the stability of nonlinear systems.  
Unlike linearization methods (which only provide local insights), Lyapunov’s method allows us to reason about **global stability** properties without explicitly solving the system equations.

**<ins>Key Ideas</ins>**

- **Lyapunov’s Direct Method**:  
  Provides a systematic way to prove stability and convergence for nonlinear systems by constructing a scalar "energy-like" function.

- **Definiteness of Functions**:  
  The concept of *positive definite*, *negative definite*, and *semi-definite* functions is introduced.  
  This serves as the **building block** for Lyapunov’s direct method.

- **Candidate Lyapunov Functions**:  
  Convenient prototypes are presented for:
  - **Rate-error measures** (functions of angular velocity error).  
  - **State-error measures** (functions of attitude error).
 
**<ins>Key Ideas</ins>**

By the end of this week, you should be able to:
1. Apply Lyapunov’s direct method to argue **stability and convergence** on a range of dynamical systems.
2. Differentiate between a range of **nonlinear stability concepts** (e.g., stability, asymptotic stability, exponential stability).

---

In [1]:
import sympy as sp
sp.init_printing(use_latex='mathjax')

from IPython.display import display, Math

import numpy as np

# 2.1 - Lyapunov's Direct Method

## 2.1.1 - Definiteness of Functions

**<ins>Definiteness of a Scalar Function</ins>**

Let $x_r$ be an equilibrium and $B_\delta(x_r)$ a ball of radius $\delta > 0$.

- **Positive definite (PD) about $x_r$**  
  $V(x_r) = 0$ and $V(x) > 0$ for all $x \in B_\delta(x_r) \setminus \{x_r\}$.

- **Positive semi-definite (PSD) about $x_r$**  
  $V(x_r) = 0$ and $V(x) \ge 0$ for all $x \in B_\delta(x_r)$  
  (zero may occur at other points besides $x_r$).

- **Negative definite (ND) about $x_r$**  
  $V(x_r) = 0$ and $V(x) < 0$ for all $x \in B_\delta(x_r) \setminus \{x_r\}$.

- **Negative semi-definite (NSD) about $x_r$**  
  $V(x_r) = 0$ and $V(x) \le 0$ for all $x \in B_\delta(x_r)$.

- **Indefinite**  
  $V(x)$ takes both positive and negative values in every neighborhood of $x_r$.

> “Local” means these conditions hold for some $\delta > 0$.  
> “Global” replaces $B_\delta(x_r)$ by $\mathbb{R}^n$.

**<ins>Matrix Definiteness (via Quadratic Form)</ins>**

A matrix $[K]$ is said to be positive or negative (semi-)definite if for every state vector $x$:

$$
x^T[K]x
\begin{cases}
> 0 & \text{positive definite} \\
\geq 0 & \text{positive semi-definite} \\
< 0 & \text{negative definite} \\
\leq 0 & \text{negative semi-definite}
\end{cases}
$$

If neither sign condition holds, $K$ is **indefinite**.

(Equivalently, check eigenvalues: all $>0$, all $\ge0$, all $<0$, all $\le0$, or mixed.)

**<ins>The Intuition</ins>**

- PD / ND: strict sign away from $x_r$; only zero at $x_r$.  
- PSD / NSD: non-strict sign; can be zero on a set containing $x_r$.  
- Indefinite: switches sign nearby.m point.

**Examples**

| Function | Type |
|-----------|------|
| $V(x) = x^\top x$ | Positive definite |
| $V(x) = x_1^2 x_2^2$ | Positive semi-definite |
| $V(x) = -x^\top x$ | Negative definite |
| $V(x) = x_1^2 - x_2^2$ | Indefinite |

In [2]:
def check_definiteness(obj, variables=None, equilibrium=None, sample_points=100):
    """
    Check definiteness of a matrix (NumPy) or a function (SymPy).
    
    Parameters
    ----------
    obj : numpy.ndarray or sympy.Expr
        The matrix or scalar function to check.
        
    variables : list of sympy.Symbol, optional
        Variables used in the function (only needed if obj is SymPy expression).
        
    equilibrium : list/tuple of floats, optional
        Equilibrium point around which to test (default = all zeros).
        
    sample_points : int
        Number of random samples near equilibrium for numeric testing (function case).
    
    Returns
    -------
    str : Definiteness classification
    """
    
    # Case 1: Matrix
    if isinstance(obj, np.ndarray):
        eigvals = np.linalg.eigvals(obj)
        
        print("Matrix:\n", obj)
        print(f"Eigenvalues: {eigvals}")
        
        if np.all(eigvals > 0):
            return "Positive definite"
        elif np.all(eigvals >= 0):
            return "Positive semi-definite"
        elif np.all(eigvals < 0):
            return "Negative definite"
        elif np.all(eigvals <= 0):
            return "Negative semi-definite"
        else:
            return "Indefinite"
    
    # Case 2: Symbolic function (expressed using SymPy)
    elif isinstance(obj, sp.Expr):
        display("Function V(x):", obj)
        
        if variables is None:
            raise ValueError("Must provide variables for SymPy function")
        if equilibrium is None:
            equilibrium = [0]*len(variables)
        
        # Check equilibrium point
        V_eq = obj.subs(dict(zip(variables, equilibrium)))
        if V_eq != 0:
            return f"Not a valid Lyapunov candidate (V(eq) = {V_eq})"
        
        # Sample near equilibrium
        vals = []
        for _ in range(sample_points):
            rand_point = np.random.uniform(-1, 1, len(variables)) * 0.5
            V_val = float(obj.subs(dict(zip(variables, rand_point))))
            vals.append(V_val)
        
        if all(v > 0 for v in vals):
            return "Positive definite (numerical check)"
        elif all(v >= 0 for v in vals):
            return "Positive semi-definite (numerical check)"
        elif all(v < 0 for v in vals):
            return "Negative definite (numerical check)"
        elif all(v <= 0 for v in vals):
            return "Negative semi-definite (numerical check)"
        else:
            return "Indefinite (numerical check)"
    
    else:
        raise TypeError("Input must be a NumPy matrix or SymPy expression")

# Example Usage:

# Case 1: Inputting a matrix
print("Case 1:")
K = np.array([[-1, 0], 
              [0, 0]])
print(check_definiteness(K))

print()

#Case 2: Inputting a function V(x)
print("Case 2:")
x, xdot = sp.symbols('x xdot')
V = 0.5*x**2 #+ 0.5*xdot**2
print(check_definiteness(V, variables=[x, xdot]))

Case 1:
Matrix:
 [[-1  0]
 [ 0  0]]
Eigenvalues: [-1.  0.]
Negative semi-definite

Case 2:


'Function V(x):'

     2
0.5⋅x 

Positive definite (numerical check)


In [3]:
# Concept Check 1 - Qn1
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 + x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

      2         2
0.5⋅x₁  + 0.5⋅x₂ 

Positive definite (numerical check)


In [4]:
# Concept Check 1 - Qn2
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 - x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

      2         2
0.5⋅x₁  - 0.5⋅x₂ 

Indefinite (numerical check)


In [5]:
# Concept Check 1 - Qn3
x1, x2 = sp.symbols('x1 x2')
V = sp.log(1 + x1**2 + x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

   ⎛  2     2    ⎞
log⎝x₁  + x₂  + 1⎠

Positive definite (numerical check)


In [6]:
# Concept Check 1 - Qn4
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 + 4*x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

      2         2
0.5⋅x₁  + 2.0⋅x₂ 

Positive definite (numerical check)


In [7]:
# Concept Check 1 - Qn5
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 + 4*x2**2) * sp.exp(-(x1**2 + 4*x2**2))
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

                         2       2
⎛      2         2⎞  - x₁  - 4⋅x₂ 
⎝0.5⋅x₁  + 2.0⋅x₂ ⎠⋅ℯ             

Positive definite (numerical check)


In [8]:
# Concept Check 1 - Qn6
K = np.array([[1.53947, -0.0422688, -0.190629], 
              [-0.0422688, 1.4759, 0.459006],
              [-0.190629, 0.459006, 1.48463]])
print(check_definiteness(K))

Matrix:
 [[ 1.53947   -0.0422688 -0.190629 ]
 [-0.0422688  1.4759     0.459006 ]
 [-0.190629   0.459006   1.48463  ]]
Eigenvalues: [1.99999822 1.50000363 0.99999815]
Positive definite


In [9]:
# Concept Check 1 - Qn7
K = np.array([[-0.984331, -1.10006, -0.478579], 
              [-1.10006, 1.03255, 0.338318],
              [-0.478579, 0.338318, 1.45178]])
print(check_definiteness(K))

Matrix:
 [[-0.984331 -1.10006  -0.478579]
 [-1.10006   1.03255   0.338318]
 [-0.478579  0.338318  1.45178 ]]
Eigenvalues: [-1.49999901  1.99999849  0.99999952]
Indefinite


In [10]:
# Concept Check 1 - Qn8
K = np.array([[-2.0353, 0.296916, -0.365128], 
              [0.296196, -1.10369, -0.074481],
              [-0.365128, -0.074481, -2.86101]])
print(check_definiteness(K))

Matrix:
 [[-2.0353    0.296916 -0.365128]
 [ 0.296196 -1.10369  -0.074481]
 [-0.365128 -0.074481 -2.86101 ]]
Eigenvalues: [-2.99999716 -1.9997948  -1.00020804]
Negative definite


## 2.1.2 - Lyapunov Function Definition

**<ins>Defintion</ins>**

A **Lyapunov function** for the nonlinear dynamical system

$$
\dot{x} = f(x)
$$

is a scalar function $V(x)$ if there exists a $\delta > 0$ such that for all  
$x \in B_\delta(x_r)$ (the neighborhood around the equilibrium $x_r$):

1. **Positive definite:**  
   $V(x)$ is positive definite about $x_r$, i.e.  
   $V(x_r) = 0$ and $V(x) > 0$ for all $x \neq x_r$ in $B_\delta(x_r)$.

2. **Smoothness:**  
   $V(x)$ has continuous partial derivatives.

3. **Lyapunov rate:**  
   The time derivative of $V(x)$ is negative semi-definite:  
   $$
   \dot{V}(x) = \frac{\partial V^T}{\partial x}\,\dot{x}
               = \frac{\partial V^T}{\partial x}\,f(x) \leq 0
   $$


**<ins>Geometric Interpretation</ins>**

<div align="center">
  <img src="Images/Wk2_EnergyBowl.PNG" alt="Energy bowl" width="500"/>
</div>

- $V(x)$ acts like an **energy bowl** centered at $x_r$.  
- The condition $\dot{V}(x) \leq 0$ means the system’s “energy” can only
  decrease or stay the same — trajectories roll downhill or orbit around a
  level set, but never climb uphill.  
- This traps trajectories inside a bounded region around $x_r$.  
- If $\dot{V}(x) = 0$, motion stays on a “rim” (like a conservative oscillator).  
- If $\dot{V}(x) < 0$, the system continuously loses energy and converges to $x_r$.

**<ins>Physical Intuition</ins>**

- A Lyapunov function is often **inspired by energy** (e.g., kinetic + potential),  
  but it doesn’t have to be physical energy — any mathematical function with  
  the three properties works.  
- Example: For a spring–mass system with no damping,  
  $V(x, \dot{x}) = \tfrac{1}{2} m \dot{x}^2 + \tfrac{1}{2} kx^2$  
  is positive definite, but $\dot{V} = 0$.  
  → The system is Lyapunov stable (oscillates with bounded energy) but not
  asymptotically stable (it doesn’t converge to the equilibrium).  

**<ins>Lyapunov Stability</ins>**

If a Lyapunov function $V(x)$ exists satisfying the three conditions above,  
then the equilibrium point $x_r$ is **stable**.

- If $\dot{V}(x) \leq 0$ → system is **Lyapunov stable**.  
- If $\dot{V}(x) < 0$ → system is **asymptotically stable** (trajectories converge to $x_r$).

## 2.1.3 - Asymptotic Stability (via Lyapunov Function)

**<ins>Definition</ins>**

Assume $V(x)$ is a Lyapunov function for the nonlinear system
$$
\dot{x} = f(x).
$$
Then the equilibrium $x_r$ is **asymptotically stable** if:

1) The system is **Lyapunov stable** about $x_r$ (i.e., $V$ is positive definite and $\dot V \le 0$ in a neighborhood $B_\delta(x_r)$), **and**  
2) $\dot V(x)$ is **negative definite** about $x_r$ (strictly $<0$ for all $x\neq x_r$ in $B_\delta(x_r)$).

Intuition: $\dot V<0$ rules out “flat rims” where the motion can orbit forever; trajectories must **lose energy** until they reach $x_r$ (as $t\to\infty$).


**<ins>Higher-Derivative Test</ins>**

Let
$$
\Omega \;=\; \{\, x \;|\; \dot V(x)=0 \,\}
$$
be the (non-empty) set where the first derivative vanishes.  
If, on $\Omega$,
- $ \dfrac{d^i V(x)}{dt^i} = 0$ for $i = 1,2,\dots,k-1$, and  
- $ \dfrac{d^{k} V(x)}{dt^{k}} < 0$ for all $x\in \Omega$,

then the system is **asymptotically stable** when **$k$ is odd**.

Intuition: if $\dot V$ is only semi-definite, look at higher time-derivatives along the “flat” set $\Omega$. The first **non-zero** derivative must be **negative** (and of odd order) to guarantee decay toward $x_r$.

> Practical recipe:  
> 1) Find a valid $V(x)$ (PD, smooth).  
> 2) Show $\dot V \le 0$ (stability).  
> 3) Identify $\Omega=\{x:\dot V=0\}$.  
> 4) Evaluate $V^{(i)}$ on $\Omega$ until the first non-zero appears.  
> 5) If that derivative is negative definite and its order is odd → asymptotic stability.


**<ins>Example: Spring–Mass–Damper</ins>**

Dynamics:
$$
m\ddot x + c\dot x + kx = 0,\qquad m>0,\; c>0,\; k>0.
$$

Candidate (total energy):
$$
V(x,\dot x) = \tfrac{1}{2}m\dot x^{2} + \tfrac{1}{2}k x^{2}.
$$

1) **First derivative**
$$
\dot V = m\dot x\,\ddot x + kx\,\dot x
       = \dot x\,(m\ddot x + kx)
       = \dot x\,(-c\dot x)
       = -c\,\dot x^{2} \;\le\; 0.
$$
→ **Stable**, but only **semi-definite** (flat whenever $\dot x=0$).

Set where $\dot V=0$:
$$
\Omega = \{\,x:\dot x=0\,\}.
$$

2) **Second derivative** (evaluate on $\Omega$)
$$
\ddot V = -2c\,\dot x\,\ddot x \quad\Rightarrow\quad
\ddot V\big|_{\Omega} = 0.
$$

3) **Third derivative** (evaluate on $\Omega$)  
Start from $\ddot V = -2c\,\dot x\,\ddot x$:
$$
\dddot V = -2c\left(\ddot x^{2} + \dot x\,\dddot x\right)
\;\Rightarrow\;
\dddot V\big|_{\Omega} = -2c\,\ddot x^{2}.
$$
On $\Omega$, the dynamics give $m\ddot x + kx = 0 \Rightarrow \ddot x = -\frac{k}{m}x$, hence
$$
\dddot V\big|_{\Omega}
= -2c\left(\frac{k^2}{m^2}\right) x^{2}
< 0 \quad \text{for } x\neq 0.
$$

- The first non-zero derivative on $\Omega$ is the **third** (odd) and it is **negative definite** in $x$.  
- By the Higher Derivative Test criterion, the equilibrium is **asymptotically stable**.

**Key takeaways**
- With **no damping** ($c=0$): $\dot V\equiv 0$ → **Lyapunov stable** (bounded oscillations), not asymptotically stable.  
- With **damping** ($c>0$): $\dot V=-c\dot x^2\le 0$ and the **third derivative test** certifies **asymptotic stability**.


**<ins>Implementation Notes (controls workflow)</ins>**
- Choose a **state vector** that contains all states that must converge (e.g., for 2nd-order mechanics, include both position and rate).  
- Reuse good Lyapunov candidates across related systems; modify the **dynamics/control** to make $\dot V$ (or a higher derivative on $\Omega$) **strictly negative**.  
- If you cannot show $\dot V<0$, try the **higher-derivative test** (or LaSalle’s Invariance Principle) to close the gap between stability and convergence.

In [11]:
# Concept Check 2 - Qn2

# Define symbols
t, k = sp.symbols('t k', positive=True, real=True)
x = sp.Function('x')(t)
xdot = sp.diff(x, t)
xddot = sp.diff(x, t, 2)

# Candidate Lyapunov function
V = sp.Rational(1,2)*xdot**2 + k*sp.Rational(1,4)*x**4
display(Math(r"V(x,\dot{x}) = " + sp.latex(V)))

# Compute Vdot = dV/dx * xdot + dV/dxdot * xddot
Vdot = sp.diff(V, x)*xdot + sp.diff(V, xdot)*xddot

# Substitute system dynamics: xddot = -k*x^3
Vdot_sub = sp.simplify(Vdot.subs(xddot, -k*x**3))
display(Math(r"\dot{V}(x,\dot{x}) = " + sp.latex(Vdot_sub)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:
# Concept Check 3 - Qn1
x, xdot, alpha = sp.symbols('x xdot alpha', real=True, positive=True)
Vdot = -alpha * xdot**2
display(Math(r"\dot{V}(x,\dot{x}) = " + sp.latex(Vdot)))

Vdot_sub = Vdot.subs(alpha, 1)
print(check_definiteness(Vdot_sub, variables=[x, xdot], equilibrium=[0, 0]))

<IPython.core.display.Math object>

'Function V(x):'

  2
-ẋ 

Negative definite (numerical check)


In [ ]:
# Concept Check 3 - Qn2
x, xdot = sp.symbols('x xdot', real=True, positive=True)
Vdot = -alpha * xdot**2
display(Math(r"\dot{V}(x,\dot{x}) = " + sp.latex(Vdot)))

Vdot_sub = Vdot.subs(alpha, 1)
print(check_definiteness(Vdot_sub, variables=[x, xdot], equilibrium=[0, 0]))